In [1]:
from datetime import datetime

import pandas as pd
import altair as alt

Loading files into pd.DataFrame objects and cleaning data

In [2]:
# Read csv files into pd.DataFrame objects

daily_activity_df = pd.read_csv("data_daily_activity.csv")

in_app_purchase_df = pd.read_csv("data_in_app_purchases.csv")

matches_df = pd.read_csv("data_matches.csv")

virtual_purchases_df = pd.read_csv("data_virtual_purchases.csv")

In [3]:
# Check data types of each column of DataFrames

# daily_activity_df.dtypes

# in_app_purchase_df.dtypes

# matches_df.dtypes
# TODO: matches_df["finish_position"] has empty values - we leave them in, as they make
# up 7% of entries

# virtual_purchases_df.dtypes


In [4]:
matches_df["finish_position"].value_counts(dropna=False)

finish_position
4.0    113190
1.0    112013
3.0    111311
2.0    103269
NaN      3597
Name: count, dtype: int64

In [5]:
# Assign correct data types to each column in each DataFrame

daily_activity_df = daily_activity_df.astype(
    {"user_id": "int64", "activity_date": "datetime64[ns]"})

in_app_purchase_df = in_app_purchase_df.astype(
    {"activity_date": "datetime64[ns]"})

matches_df = matches_df.astype(
    {"user_id": "int64",  "activity_date": "datetime64[ns]", "finish_position": "object"})

virtual_purchases_df = virtual_purchases_df.astype(
    {"activity_date": "datetime64[ns]"})

In [6]:
# daily_activity_df['platform'] has duplicate values, with different case-letters. We
# make it more uniform by applying str.title()

daily_activity_df['platform'] = daily_activity_df['platform'].str.title()

daily_activity_df['platform'].value_counts()

platform
Android    676567
Ios         81217
Name: count, dtype: int64

(Q) Which players are most profitable? (ie. which players have the highest in-app lifetime purchase amount?)

In [7]:
# Acquire all unique user_id values

unique_user_ids = in_app_purchase_df["user_id"].drop_duplicates()

unique_user_ids.sort_values()

4029    272143
2676    272161
957     272164
55      272189
646     272192
         ...  
279     309220
1634    309232
1026    309279
205     309320
70      309395
Name: user_id, Length: 1042, dtype: int64

In [8]:
# Start constructing relevant pd.DataFrame object.
# This code sums all purchases made by a given player, and allocates values into a dict.

n = len(unique_user_ids)

total_purchase_dict = {}

for i in range(n):

    iter_user_id = unique_user_ids.iloc[i]

    total_purchase_value = in_app_purchase_df[in_app_purchase_df['user_id'] == iter_user_id]['dollar_purchase_value'].sum()

    total_purchase_dict[iter_user_id] = total_purchase_value.round(2)

total_purchase_dict

    

{np.int64(299207): np.float64(29.94),
 np.int64(308639): np.float64(9.98),
 np.int64(292230): np.float64(334.53),
 np.int64(300371): np.float64(34.93),
 np.int64(278774): np.float64(34.93),
 np.int64(280995): np.float64(24.95),
 np.int64(304552): np.float64(109.84),
 np.int64(300924): np.float64(54.91),
 np.int64(308696): np.float64(9.98),
 np.int64(303304): np.float64(29.94),
 np.int64(294418): np.float64(4.99),
 np.int64(288936): np.float64(139.83),
 np.int64(295203): np.float64(119.77),
 np.int64(288480): np.float64(29.94),
 np.int64(304606): np.float64(74.9),
 np.int64(288709): np.float64(39.95),
 np.int64(304317): np.float64(19.96),
 np.int64(305762): np.float64(9.98),
 np.int64(284891): np.float64(109.78),
 np.int64(299302): np.float64(34.94),
 np.int64(282798): np.float64(758.48),
 np.int64(275614): np.float64(79.84),
 np.int64(284707): np.float64(314.63),
 np.int64(283035): np.float64(4.99),
 np.int64(279199): np.float64(1240.72),
 np.int64(304386): np.float64(44.92),
 np.int64

In [9]:
# Assemble components to form pd.DataFrame.

player_purchase_df = pd.DataFrame({"user_id": total_purchase_dict.keys(),
                                   "total_lifetime_purchase": total_purchase_dict.values()})

player_purchase_df.sort_values(by="total_lifetime_purchase", ascending=False).reset_index(drop=True)

,user_id,total_lifetime_purchase
0,279199,1240.72
1,282798,758.48
2,292230,334.53
3,284707,314.63
4,279328,309.67
...,...,...
1037,302027,4.99
1038,282351,4.99
1039,303822,4.99
1040,306661,0.00


(Q) What does the distribution of values for in-app lifetime purchase values look like for players?

In [10]:
# Create function to get cumulative frequency of purchases for players

def get_cum_freq(input_series: pd.Series):

    n = len(input_series)

    result = []

    for i in range(n):
        if i == 0:
            result.append(input_series.iloc[i])
        else:
            result_sum = result[i - 1] + input_series.iloc[i]
            result.append(result_sum)

    return result

In [11]:
# Add column for cumulative frequency

player_purchase_df = player_purchase_df.sort_values(
    by="total_lifetime_purchase", ascending=False).reset_index(drop=True)

player_purchase_df['rank'] = player_purchase_df.index + 1

player_purchase_df["cum_freq_purchase"] = get_cum_freq(player_purchase_df['total_lifetime_purchase'])

player_purchase_df.rename(columns={'rank': 'Players lifetime purchase ranking (Highest to lowest)',
                                   'cum_freq_purchase': 'Cumulative Purchase Quantity'}, inplace=True)

player_purchase_df

,user_id,total_lifetime_purchase,Players lifetime purchase ranking (Highest to lowest),Cumulative Purchase Quantity
0,279199,1240.72,1,1240.72
1,282798,758.48,2,1999.20
2,292230,334.53,3,2333.73
3,284707,314.63,4,2648.36
4,279328,309.67,5,2958.03
...,...,...,...,...
1037,302027,4.99,1038,25064.20
1038,282351,4.99,1039,25069.19
1039,303822,4.99,1040,25074.18
1040,306661,0.00,1041,25074.18


In [12]:
base = alt.Chart(player_purchase_df,
                 title='Distribution of Lifetime Purchase Value')

chart = base.mark_area().encode(alt.Y("Cumulative Purchase Quantity:Q"),
                                alt.X(
                                    "Players lifetime purchase ranking (Highest to lowest):Q")
                                ).interactive()

chart

alt.Chart(...)

In [13]:
agg_lifetime_purchase = player_purchase_df['total_lifetime_purchase'].sum()

quarter_y_rule = base.mark_rule(color="red", strokeDash=[2, 2]).encode(
    y=alt.datum(agg_lifetime_purchase* 0.25)
)

half_y_rule = base.mark_rule(color="red", strokeDash=[2, 2]).encode(
    y=alt.datum(agg_lifetime_purchase * 0.5)
)

three_quarter_y_rule = base.mark_rule(color="red", strokeDash=[2, 2]).encode(
    y=alt.datum(agg_lifetime_purchase * 0.75)
)

quarter_x_rule = base.mark_rule(color="red", strokeDash=[2, 2]).encode(
    x=alt.datum(23.03)).interactive()

half_x_rule = base.mark_rule(color="red", strokeDash=[2, 2]).encode(
    x=alt.datum(101.595)).interactive()

three_quarter_x_rule = base.mark_rule(color="red", strokeDash=[2, 2]).encode(
    x=alt.datum(301.96)).interactive()

chart + quarter_y_rule + half_y_rule + three_quarter_y_rule + quarter_x_rule + half_x_rule + three_quarter_x_rule

alt.LayerChart(...)

Conclusion:

• The top 24 players with largest purchases make up over 25% of aggregated lifetime purchases

• The top 102 players make up over 50%

• The top 302 make up over 75%

Actions:

• Analyse top 24 highest-purchasing players

We firstly check for repeat purchases amongst the top 24 highest in-app purchasing players

In [14]:
top_in_app_purchasing_players_id = player_purchase_df.iloc[0:24]['user_id']

top_in_app_purchasing_players_id

0     279199
1     282798
2     292230
3     284707
4     279328
5     299527
6     280804
7     276293
8     305048
9     299270
10    306713
11    295235
12    274469
13    282387
14    309213
15    290698
16    283220
17    299625
18    298847
19    299427
20    284622
21    287037
22    307559
23    288936
Name: user_id, dtype: int64

In [28]:
top_in_app_purchasing_players_id_ls = top_in_app_purchasing_players_id.to_list()

in_app_purchase_df_top_players = in_app_purchase_df[in_app_purchase_df['user_id'].isin(
    top_in_app_purchasing_players_id_ls)]

# in_app_purchase_df_top_players_date = in_app_purchase_df_top_players[['user_id', 'activity_date', 'product_group', 'purchase_number']]

in_app_purchase_df_top_players['product_group'].value_counts()

product_group
Gold          585
Promotion     230
Remove Ads     70
Name: count, dtype: int64

In [26]:
top_in_app_purchasing_players_id_ls

[279199,
 282798,
 292230,
 284707,
 279328,
 299527,
 280804,
 276293,
 305048,
 299270,
 306713,
 295235,
 274469,
 282387,
 309213,
 290698,
 283220,
 299625,
 298847,
 299427,
 284622,
 287037,
 307559,
 288936]

In [27]:
result = {}

for user_id in top_in_app_purchasing_players_id_ls:
    
    user_id_repeat = len(in_app_purchase_df_top_players[in_app_purchase_df_top_players['user_id'] == user_id])

    if user_id_repeat > 1:

        result[user_id] = user_id_repeat

result

{279199: 128,
 282798: 152,
 292230: 47,
 284707: 37,
 279328: 33,
 299527: 49,
 280804: 19,
 276293: 37,
 305048: 24,
 299270: 36,
 306713: 15,
 295235: 20,
 274469: 29,
 282387: 25,
 309213: 23,
 290698: 28,
 283220: 32,
 299625: 31,
 298847: 12,
 299427: 26,
 284622: 30,
 287037: 19,
 307559: 16,
 288936: 17}

TO-DO:

• Generate scatter graph with no. of purchases on x-axis, and lifetime purchase value in y-axis => highlights that people with more repeat purchase generate higher lifetime purchase value

• Generate line graph with activity date on x-axis, no. of purchases on y-axis, and colour of line as user_id => detect any sudden spikes/drops in purchasing activity

    • Possibly create separate chart, where dates are grouped by weekday, to detect trends in in-app purchases

• Generate pie chart with theta as no. of purchases, and colour as product_group => highlights the most popular product group